In [125]:
from lxml import html
from bs4 import BeautifulSoup
import requests
import re

In [126]:
main_url = 'https://ro.wikipedia.org'
url = main_url + '/wiki/Categorie:Liste_de_localități_din_România'
page = requests.get(url, timeout=1)

In [127]:
print(page.content)

b'<!DOCTYPE html>\n<html class="client-nojs" lang="ro" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Categorie:Liste de localit\xc4\x83\xc8\x9bi din Rom\xc3\xa2nia - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Categorie:Liste_de_localit\xc4\x83\xc8\x9bi_din_Rom\xc3\xa2nia","wgTitle":"Liste de localit\xc4\x83\xc8\x9bi din Rom\xc3\xa2nia","wgCurRevisionId":10071500,"wgRevisionId":10071500,"wgArticleId":753352,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Liste de localit\xc4\x83\xc8\x9bi dup\xc4\x83 \xc8\x9bar\xc4\x83","Liste despre geografia Rom\xc3\xa2niei","Localit\xc4\x83\xc8\x9bi \xc3\xaen Rom\xc3\xa2nia"],"wgBreakFrames

In [128]:
soup = BeautifulSoup(page.text, "html.parser")
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="ro">
<head>
<meta charset="utf-8"/>
<title>Categorie:Liste de localități din România - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Categorie:Liste_de_localități_din_România","wgTitle":"Liste de localități din România","wgCurRevisionId":10071500,"wgRevisionId":10071500,"wgArticleId":753352,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Liste de localități după țară","Liste despre geografia României","Localități în România"],"wgBreakFrames":false,"wgPageContentLanguage":"ro","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t.",".\t,"]

In [129]:
all_links = soup.find_all("div", class_="mw-category")[0].find_all("a")
pages_links = []
for l in all_links:
    if "județul" in l.text:
        pages_links.append(l.get("href"))
        
print(len(pages_links))

41


In [156]:
def parse_jud(soup_jud):
    title = soup_jud.find("h1", id="firstHeading").text
#     print(title)
    jud_name = title.split("județul ", 1)[1]
    return jud_name

In [131]:
# jud_url = "https://ro.wikipedia.org/wiki/List%C4%83_de_localit%C4%83%C8%9Bi_din_jude%C8%9Bul_Satu_Mare"
# page_jud = requests.get(jud_url, timeout=1)
# soup_jud = BeautifulSoup(page_jud.text, "html.parser")

In [132]:
# title = soup_jud.find("h1", id="firstHeading").text
# print(title)

In [133]:
# jud = re.match(r'județul (\w+)', title, re.I|re.M)

In [157]:
rows_list = []

for jud_page in pages_links:
    jud_url = main_url+jud_page
    page_jud = requests.get(jud_url, timeout=1)
    soup_jud = BeautifulSoup(page_jud.text, "html.parser")
    jud_name = parse_jud(soup_jud)
    tabl = soup_jud.find("table", class_="wikitable sortable")
    for row in tabl.findAll("tr"):
        cell = row.find("td")
        if cell:
            cell_link = cell.find("a")
            if cell_link:
                nume_localitate = cell_link.text
                rows_list.append({'Judet':jud_name, 'Localitate':nume_localitate})

import pandas as pd
x = pd.DataFrame(rows_list)
print(x.size)


26410


In [167]:
print(x.loc[0])

Judet          Alba
Localitate    Abrud
Name: 0, dtype: object


In [166]:
x.to_csv("localitati_ro_big.csv")